# ZEC XMR Prototype

In [8]:
# Imports
import pandas as pd
from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py
import datetime
import json
import math

import schedule
import time
from datetime import datetime, timedelta
from joblib import dump, load

import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import clear_output

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)

In [78]:
def get_margin_asset(name:str):
    """returns a dictionary with:
    - asset name
    - free
    - locked
    - borrowed
    - interest
    - netAsset"""
    return list(filter(lambda x: x['asset'] == name, client.get_margin_account()["userAssets"]))[0]

def binance_ceil(x:float):
    """returns the ceil to 5 decimal places (to payback borrowed amounts). Includes 0.1% trading fee"""
    return math.ceil(x*1.001*100000)/100000

def binance_floor(x:float):
    """returns the floor to 5 decimal places amount not including trading fee. (for liquidating)"""
    return math.floor(x*100000)/100000

def go_long_short(long: str, l_amt: float, short: str, s_amt: float, base="USDT"):
    """goes long and short. Enter long/short as 'XMR' rather than 'XMRUSDT'. """
    l_pair = long + base
    s_pair = short + base
    transaction = client.create_margin_loan(asset=short, amount=str(s_amt))
    order_s = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL")
    order_l = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL")
    send_message("ls", long=long, l_amt=str(l_amt), short=short, s_amt=str(s_amt), o_s=order_s, o_l=order_l)
    
def liquidate(long: str, short: str, base="USDT"):
    """liquidates long and short position. Enter long as currently longing which asset"""
    l_pair = long + base
    s_pair = short + base
    l_amt = binance_floor(float(get_margin_asset(long)["free"]))
    s_amt = binance_ceil(abs(float(get_margin_asset(short)["netAsset"])))
    
    order = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL")
    order1 = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL")
    rp = str(abs(float(get_margin_asset(short)["free"])))
    transaction1 = client.repay_margin_loan(asset=short, amount=rp)
    send_message("lq", o_s=order, o_b=order1)

    
def get_price(symbol:str):
    """returns the price. symbol MUST include USDT, ie ZECUSDT"""
    return float(client.get_recent_trades(symbol=symbol, limit=1)[0]["price"])

def trade_amt(symbol:str, total:float):
    """returns the amount of symbol to trade to be half of total. Note, symbol MUST include USDT"""
    p = get_price(symbol)
    t = total/2
    return binance_ceil(t/p)

def get_long():
    """Returns None if long nothing, else 'ZEC' or 'XMR'"""
    if float(get_margin_asset("ZEC")["borrowed"])>0: #Borrowed ZEC (ie long XMR)
        return "XMR"
    elif float(get_margin_asset("XMR")["borrowed"])>0: #Borrwed XMR (ie long ZEC)
        return "ZEC"
    else:
        return None

In [24]:
get_margin_asset("USDT")

{'asset': 'USDT',
 'free': '29.94558741',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '29.94558741'}

In [25]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.00000367',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '0.00000367'}

In [26]:
transaction = client.create_margin_loan(asset='ZEC', amount='0.15')
order = client.create_margin_order(
    symbol='ZECUSDT',
    side=SIDE_SELL,
    type=ORDER_TYPE_MARKET,
    quantity=0.15)

In [27]:
get_margin_asset("USDT")

{'asset': 'USDT',
 'free': '44.84067741',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '44.84067741'}

In [28]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.00000367',
 'locked': '0',
 'borrowed': '0.15',
 'interest': '0.00000188',
 'netAsset': '-0.14999821'}

In [29]:
q = abs(float(get_margin_asset("ZEC")["netAsset"]))
q

0.14999821

In [32]:
q = binance_ceil(q)
q

0.15015

In [33]:
order1 = client.create_margin_order(
    symbol='ZECUSDT',
    side=SIDE_BUY,
    type=ORDER_TYPE_MARKET,
    quantity=q)

In [34]:
get_margin_asset("USDT")

{'asset': 'USDT',
 'free': '29.90075241',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '29.90075241'}

In [35]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.15000352',
 'locked': '0',
 'borrowed': '0.15',
 'interest': '0.00000188',
 'netAsset': '0.00000164'}

In [36]:
q = abs(float(get_margin_asset("ZEC")["free"]))
q

0.15000352

In [37]:
transaction1 = client.repay_margin_loan(asset='ZEC', amount=str(q))

In [38]:
get_margin_asset("ZEC")

{'asset': 'ZEC',
 'free': '0.00003167',
 'locked': '0',
 'borrowed': '0',
 'interest': '0',
 'netAsset': '0.00003167'}

In [9]:
float(client.get_recent_trades(symbol='XMRUSDT', limit=1)[0]["price"])

196.2

In [120]:
def printer():
    """main printer. Fetches current position, fetches data, and performs buys/sells if necessary"""
    
    long = get_long() # Returns None if long nothing, else "ZEC" or "XMR"
    
    #total will be ignored if in liquidating phase
    #-10 for creating a buffer
    total = float(get_margin_asset("USDT")["free"])-10 
    
    thres = 1.        
    sell_thres = 1.   
    
    # Get z_score
    z = get_z_score()
    while math.isnan(z): #z is nan, retry:
        time.sleep(1)
        z = get_z_score()
    
    # Get trading amount (won't be used if liquidating or NOT buying)
    zec = trade_amt("ZECUSDT", total)
    xmr = trade_amt("XMRUSDT", total)
    
    if z>thres and long==None: #buy/sell #LONG ZEC
        go_long_short(long="ZEC", l_amt=zec, short="XMR", s_amt=xmr)
    elif z<-thres and long==None: #buy/sell #LONG XMR
        go_long_short(long="XMR", l_amt=xmr, short="zec", s_amt=zec)
    elif z>sell_thres and long=="XMR": #liquidate #LONG XMR
        liquidate(long="XMR", short="ZEC")
    elif z<-sell_thres and long=="ZEC": #liquidate #LONG ZEC
        liquidate(long="ZEC", short="XMR")
    
    send_message("z", z=str(z))
    reset_client()

In [79]:
def reset_client():
    """resets the client to prevent 'read operation timed out'"""
    global client
    global api_key
    global api_secret
    client = Client(api_key=api_key, api_secret=api_secret)
        
def get_minutely_data(symbol:str, kline_interval:object, start="1 Jan 1900", days=3.5):
    """
    downloads binance data
    symbol: BTCBUSD
    kline_interval: Client.KLINE_INTERVAL_1DAY, Client.KLINE_INTERVAL_1DAY, Client.KLINE_INTERVAL_1DAY
    interval_name: only used for csv name: BTCUSDT-interval_name.csv
    start: empty if from the very beginning
    """
    d = datetime.today() - timedelta(days=days)
    start_date = d.strftime("%d %b %Y %H:%M:%S")
    today = datetime.today().strftime("%d %b %Y %H:%M:%S")
    klines = client.get_historical_klines(symbol, kline_interval, start_date, today, 1000)
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)
    data_latest = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    
    index = data.index[(data['open'] == data_latest.iloc[0].open) & (data['high'] == data_latest.iloc[0].high)].tolist()[0]
    result = pd.concat([data[:index], data_latest], ignore_index=True, sort=False)
    return result

def get_z_score(hr = 1.534477217835591): 
    '''gets the latest z-score, given hedge ratio hr.
    Warning, sometimes it gives nan, just rerun (binance's fault)'''
    zec = get_minutely_data("ZECUSDT", Client.KLINE_INTERVAL_1MINUTE, days=1.2)
    xmr = get_minutely_data("XMRUSDT", Client.KLINE_INTERVAL_1MINUTE, days=1.2)
    df = pd.to_numeric(zec.open.rename("A")).to_frame()
    df["B"] = pd.to_numeric(xmr.open)
    spread = pd.Series((df['B'] - hr * df['A'])).rename("spread").to_frame()
    spread["mean"] = spread.spread.rolling(2000).mean()
    spread["std"] =  spread.spread.rolling(2000).std()
    spread["zscore"] = pd.Series((spread["spread"]-spread["mean"])/spread["std"])
    return spread.iloc[-1].zscore

In [122]:
get_z_score()

-1.8892500737204911

In [13]:
schedule.clear()
schedule.every().minute.at(":01").do(printer)

Every 1 minute at 00:00:01 do printer() (last run: [never], next run: 2021-06-14 09:39:01)

In [112]:
o = client.get_all_margin_orders(symbol='ZECUSDT')[3]

In [115]:
client.get_margin_order(
    symbol='ZECUSDT',
    orderId='1022022871')

{'symbol': 'ZECUSDT',
 'orderId': 1022022871,
 'clientOrderId': 'G25bdpEChH36lZkhyB9ZUs',
 'price': '0',
 'origQty': '0.15',
 'executedQty': '0.15',
 'cummulativeQuoteQty': '14.79',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'stopPrice': '0',
 'icebergQty': '0',
 'time': 1626243894228,
 'updateTime': 1626243894228,
 'isWorking': True,
 'accountId': 129068654,
 'isIsolated': False}

In [118]:
client.get_all_margin_orders(symbol='ZECUSDT', newOrderRespType = "FULL")[0]

{'symbol': 'ZECUSDT',
 'orderId': 1022022871,
 'clientOrderId': 'G25bdpEChH36lZkhyB9ZUs',
 'price': '0',
 'origQty': '0.15',
 'executedQty': '0.15',
 'cummulativeQuoteQty': '14.79',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'stopPrice': '0',
 'icebergQty': '0',
 'time': 1626243894228,
 'updateTime': 1626243894228,
 'isWorking': True,
 'isIsolated': False}

In [119]:
# Discord # Saving message
def get_message():
    """gets the message"""
    with open('message.txt') as json_file:
        new_message = json.load(json_file)
        return new_message
def send_message(m:str,  **k):
    data = get_message()
    if m == "ls":
        data["message"] = f"Went long {k['l_amt']} {k['long']} at ${k['o_l']['fills'][0]['price']} \nShort {k['s_amt']} {k['short']} at ${k['o_s']['fills'][0]['price']}"
        data["o_l"] = k['o_l']
        data["o_s"] = k['o_s']
    elif m == "lq":
        o_s = k['o_s']
        o_b = k['o_b']
        data["message"] = f"Liquidated! Sold {o_s['executedQty']} {o_s['symbol']} for {o_s['fills'][0]['price']} \nBought {o_b['executedQty']} {o_b['symbol']} for {o_b['fills'][0]['price']}"
        data["o_l"] = o_s
        data["o_s"] = o_b
    elif m == "z":
        data["z"] = k["z"]
    with open('message.txt', 'w') as outfile:
        json.dump(data, outfile)

In [15]:
sleep = 1
while True:
    clear_output()
    schedule.run_pending()
    print(f"sleeping for {sleep} seconds")
    time.sleep(sleep)

awaiting tp and sl orders to fulfill
sleeping for 614 seconds


KeyboardInterrupt: 